In [1]:
from rdflib import Graph
from SPARQLWrapper import SPARQLWrapper, JSON, N3
from pprint import pprint

Create the Query and send it via SPARQLWrapper

In [13]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

sparql = SPARQLWrapper('https://dbpedia.org/sparql') # initialize SPARQL Wrapper

# # Example: look for an object "Barack_Obama" and get the property "label"
# sparql.setQuery('''
#     SELECT ?object
#     WHERE { dbr:Barack_Obama dbo:label ?object .}
# ''')


sparql.setQuery('''
    SELECT ?object
    # WHERE {  ?GPE rdfs:label      "Gdansk"@en }
    WHERE { dbr:Gdansk rdfs:label ?object .}
''')
sparql.setReturnFormat(JSON)

sparql.verify = False
qres = sparql.query().convert()
pprint(qres)

{'head': {'link': [], 'vars': ['object']},
 'results': {'bindings': [{'object': {'type': 'literal',
                                      'value': 'Gdansk',
                                      'xml:lang': 'en'}}],
             'distinct': False,
             'ordered': True}}


Print the results:

In [14]:
for result in qres['results']['bindings']:
    # print(result['object'])
    
    lang, value = result['object']['xml:lang'], result['object']['value']
    print(f'Lang: {lang}\tValue: {value}')
    # if lang == 'en':
    #     print(value)

Lang: en	Value: Gdansk


Get the parent and child nodes. This allows to create a graph of objects (CONSTRUCT). Parent - Artificial Intelligence, Child - for example Markov Models.

In [16]:
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery('''
CONSTRUCT { dbc:Machine_learning skos:broader ?parent .
            dbc:Machine_learning skos:narrower ?child .} 
WHERE {
    { dbc:Machine_learning skos:broader ?parent . }
UNION
    { ?child skos:broader dbc:Machine_learning . }
}''')

sparql.setReturnFormat(N3)  # N triples
qres = sparql.query().convert()

g = Graph()
g.parse(data=qres, format='n3')
print(g.serialize(format='ttl'))

@prefix dbc: <http://dbpedia.org/resource/Category:> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .

dbc:Machine_learning skos:broader dbc:Artificial_intelligence,
        dbc:Learning ;
    skos:narrower dbc:Applied_machine_learning,
        dbc:Artificial_intelligence_conferences,
        dbc:Artificial_neural_networks,
        dbc:Bayesian_networks,
        dbc:Blockmodeling,
        dbc:Classification_algorithms,
        dbc:Cluster_analysis,
        dbc:Computational_learning_theory,
        dbc:Data_mining_and_machine_learning_software,
        dbc:Datasets_in_machine_learning,
        dbc:Deep_learning,
        dbc:Dimension_reduction,
        dbc:Ensemble_learning,
        dbc:Evolutionary_algorithms,
        dbc:Genetic_programming,
        dbc:Inductive_logic_programming,
        dbc:Kernel_methods_for_machine_learning,
        dbc:Latent_variable_models,
        dbc:Learning_in_computer_vision,
        dbc:Log-linear_models,
        dbc:Loss_functions,
        dbc:

Iteration through list of entities. In each iteration we get: label text, comment text, thumbnail image.

In [36]:
from IPython.display import display, Image

sparql = SPARQLWrapper('https://dbpedia.org/sparql')

instruments = ['Air_horn', 'Kazoo', 'Mandolin', 'Clavichord', 'Kaval', 'Electronic_keyboard', 'Choghur', 'Zill']

for instrument in instruments:
    print('###########################################')
    sparql.setQuery(f'''
    SELECT ?name ?comment ?image
    WHERE {{ dbr:{instrument} rdfs:label ?name.
             dbr:{instrument} rdfs:comment ?comment.
             dbr:{instrument} dbo:thumbnail ?image.
    
        FILTER (lang(?name) = 'en')
        FILTER (lang(?comment) = 'en')
    }}''')

    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert()
    
    result = qres['results']['bindings'][0]
    name, comment, image_url = result['name']['value'], result['comment']['value'], result['image']['value']

    print(name)

    # Attempt to open the image with error handling
    try:
        print("image_url:", (image_url))
        display(Image(url=image_url, width=100, unconfined=True))
    except Exception as e:
        print(f'Error: {e}')
    
    print(f'{comment}...')


###########################################
Air horn
image_url: http://commons.wikimedia.org/wiki/Special:FilePath/Supertanker_Steam_whistle,_Merseyside_Maritime_Museum.jpg?width=300


An air horn is a pneumatic device designed to create an extremely loud noise for signaling purposes. It usually consists of a source which produces compressed air, which passes into a horn through a reed or diaphragm. The stream of air causes the reed or diaphragm to vibrate, creating sound waves, then the horn amplifies the sound making it louder. Air horns are widely employed as vehicle horns, installed on large buses, semi-trailer trucks, fire trucks, trains, and some ambulances as a warning device, and on ships as a signaling device....
###########################################
Kazoo
image_url: http://commons.wikimedia.org/wiki/Special:FilePath/Kazoo.jpg?width=300


The kazoo is an American musical instrument that adds a "buzzing" timbral quality to a player's voice when the player vocalizes into it. It is a type of mirliton (which itself is a membranophone), one of a class of instruments which modifies its player's voice by way of a vibrating membrane of goldbeater's skin or material with similar characteristics. Similar hide-covered vibrating and voice-changing instruments have been used in Africa for hundreds of years....
###########################################
Mandolin
image_url: http://commons.wikimedia.org/wiki/Special:FilePath/Paris_Swing_Samois_Mandolin_MS-130-SN.jpg?width=300


A mandolin (Italian: mandolino pronounced [mandoˈliːno]; literally "small mandola") is a stringed musical instrument in the lute family and is generally plucked with a pick. It most commonly has four courses of doubled metal strings tuned in unison, thus giving a total of 8 strings, although five (10 strings) and six (12 strings) course versions also exist. The courses are typically tuned in an interval of perfect fifths, with the same tuning as a violin (G3, D4, A4, E5). Also, like the violin, it is the soprano member of a family that includes the mandola, octave mandolin, mandocello and mandobass....
###########################################
Clavichord
image_url: http://commons.wikimedia.org/wiki/Special:FilePath/Hillclavichord.jpg?width=300


The clavichord is a stringed rectangular keyboard instrument that was used largely in the Late Middle Ages, through the Renaissance, Baroque and Classical eras.Historically, it was mostly used as a practice instrument and as an aid to composition, not being loud enough for larger performances. The clavichord produces sound by striking brass or iron strings with small metal blades called tangents. Vibrations are transmitted through the bridge(s) to the soundboard....
###########################################
Kaval
image_url: http://commons.wikimedia.org/wiki/Special:FilePath/Bulgarian_kaval_in_key_of_D,_African_blackwood.jpg?width=300


The kaval is a chromatic end-blown flute traditionally played throughout the Balkans (in Albania, Romania, Bulgaria, Southern Serbia, Kosovo, North Macedonia, Northern Greece, and elsewhere) and Anatolia (including Turkey and Armenia). The kaval is primarily associated with mountain shepherds....
###########################################
Electronic keyboard
image_url: http://commons.wikimedia.org/wiki/Special:FilePath/YamahaKeyboard-2.jpg?width=300


An electronic keyboard, portable keyboard, or digital keyboard is an electronic musical instrument, an electronic derivative of keyboard instruments. Electronic keyboards include synthesizers, digital pianos, stage pianos, electronic organs and digital audio workstations. In technical terms, an electronic keyboard is a synthesizer with a low-wattage power amplifier and small loudspeakers....
###########################################
Choghur
image_url: http://commons.wikimedia.org/wiki/Special:FilePath/Çoğur.jpg?width=300


The choghur (Azerbaijani: Çoğur; Georgian: ჩონგური) is a plucked string musical instrument common in Azerbaijan and Georgia. It has 4 nylon strings. The choghur dates back to the 12th to 16th centuries, the period between the gopuz and the Bağlama. In the Caucasus, Iran and Anatolia, and in Sufi traditions, darvishes and ashugs used an instrument called the "chaghyr" /"chagur"/ "chugur" / "choghur" / "chungur". Presumably, the name "choghur" means "the musical instrument used to appeal to God and truth". [In Azerbaijani the word "chaghir" means "to call", "to appeal"] It may be assumed that the name of the instrument originates from the expression "chal-chaghyr" (festivity or celebration), which was later changed to "choghur". Various historical sources indicate that the choghur was used t...
###########################################
Zill
image_url: http://commons.wikimedia.org/wiki/Special:FilePath/Finger_cymbals.jpg?width=300


Zills or zils (from Turkish zil 'cymbals'), also called finger cymbals, are small metallic cymbals used in belly dancing and similar performances. They are called sāgāt (‏صاجات‎) in Egypt. They are similar to Tibetan tingsha bells. In Western music, several pairs can be set in a frame to make a tambourine. Names in other languages include nuqaisāt in Arabic and used among Berbers, Zang-e sarangoshti (سنج انگشتی) in Persian, çeng in Turkish, p'eng chung in Chinese....
